In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.applications.inception_v3 import InceptionV3, preprocessinput
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.layers import Input
from keras import models
from keras import layers
from keras import optimizers
import random 
from keras.preprocessing.image import ImageDataGenerator
from random import randrange
import cv2
from keras import backend as K
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.applications.vgg19 import preprocessinput
from keras.layers import Activation, Flatten, Dropout, BatchNormalization



data = pd.read_csv('fgo_multiclass_labels.csv')

In [3]:
data.head()

data.columns

Index(['Unnamed: 0', 'full_path', 'image_name', 'name', 'white', 'red',
       'green', 'black', 'blue', 'purple', 'gold', 'silver', 'gender_Female',
       'gender_Male', 'region_Asia', 'region_Egypt', 'region_Europe',
       'region_Middle East', 'fighting_type_magic', 'fighting_type_melee',
       'fighting_type_ranged', 'alignment_CE', 'alignment_CG', 'alignment_CN',
       'alignment_LE', 'alignment_LG', 'alignment_LN', 'alignment_NE',
       'alignment_NG', 'alignment_TN'],
      dtype='object')

In [4]:
X = data['image_name']
y = data[['white', 'red',
       'green', 'black', 'blue', 'purple', 'gold', 'silver', 'gender_Female',
       'gender_Male', 'region_Asia', 'region_Egypt', 'region_Europe',
       'region_Middle East', 'fighting_type_magic', 'fighting_type_melee',
       'fighting_type_ranged', 'alignment_CE', 'alignment_CG', 'alignment_CN',
       'alignment_LE', 'alignment_LG', 'alignment_LN', 'alignment_NE',
       'alignment_NG', 'alignment_TN']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)



In [5]:
y_train.columns

Index(['white', 'red', 'green', 'black', 'blue', 'purple', 'gold', 'silver',
       'gender_Female', 'gender_Male', 'region_Asia', 'region_Egypt',
       'region_Europe', 'region_Middle East', 'fighting_type_magic',
       'fighting_type_melee', 'fighting_type_ranged', 'alignment_CE',
       'alignment_CG', 'alignment_CN', 'alignment_LE', 'alignment_LG',
       'alignment_LN', 'alignment_NE', 'alignment_NG', 'alignment_TN'],
      dtype='object')

In [6]:
X_train = X_train.values.tolist()
X_test = X_test.values.tolist()

#train
color_list = ['white', 'red',
       'green', 'black', 'blue', 'purple', 'gold', 'silver']
color_list_train = y_train[color_list]
color_list_nodes = color_list_train.shape[1]
color_list_train = color_list_train.values.tolist()

gender = ['gender_Female',
       'gender_Male']
gender_train = y_train[gender]
gender_nodes = gender_train.shape[1]
gender_train = gender_train.values.tolist()

area = ['region_Asia', 'region_Egypt', 'region_Europe',
       'region_Middle East']
area_train = y_train[area]
area_nodes = area_train.shape[1]
area_train = area_train.values.tolist()

fighting_style = ['fighting_type_magic', 'fighting_type_melee',
       'fighting_type_ranged']
fighting_style_train = y_train[fighting_style]
fighting_nodes = fighting_style_train.shape[1]
fighting_style_train = fighting_style_train.values.tolist()

align = ['alignment_CE', 'alignment_CG', 'alignment_CN',
       'alignment_LE', 'alignment_LG', 'alignment_LN', 'alignment_NE',
       'alignment_NG', 'alignment_TN']
align_train = y_train[align]
align_nodes = align_train.shape[1]
align_train = align_train.values.tolist()

print(color_list_nodes,gender_nodes,area_nodes,fighting_nodes,align_nodes)
#test
color_list = ['white', 'red',
       'green', 'black', 'blue', 'purple', 'gold', 'silver']
color_list_test = y_test[color_list]
color_list_nodes = color_list_test.shape[1]
color_list_test = color_list_test.values.tolist()

gender = ['gender_Female',
       'gender_Male']
gender_test = y_test[gender]
gender_nodes = gender_test.shape[1]
gender_test = gender_test.values.tolist()

area = ['region_Asia', 'region_Egypt', 'region_Europe',
       'region_Middle East']
area_test = y_test[area]
area_nodes = area_test.shape[1]
area_test = area_test.values.tolist()

fighting_style = ['fighting_type_magic', 'fighting_type_melee',
       'fighting_type_ranged']
fighting_style_test = y_test[fighting_style]
fighting_nodes = fighting_style_test.shape[1]
fighting_style_test = fighting_style_test.values.tolist()

align = ['alignment_CE', 'alignment_CG', 'alignment_CN',
       'alignment_LE', 'alignment_LG', 'alignment_LN', 'alignment_NE',
       'alignment_NG', 'alignment_TN']
align_test = y_test[align]
align_nodes = align_test.shape[1]
align_test = align_test.values.tolist()

print(color_list_nodes,gender_nodes,area_nodes,fighting_nodes,align_nodes)

8 2 4 3 9
8 2 4 3 9


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:


# initialize the number of epochs and batch size
EPOCHS = 50
BS = 16


def image_generator(kinglists, bs, mode="train", aug=None):
    # loop indefinitely
    
    while True:
        # initialize our batches of images and labels
        images = []
        #labels = []
        
        color_list = []
        gender = []
        area = []
        fight = []
        align = []
        
        # keep looping until we reach our batch size
        while len(images) < bs:
            combinedlabellist = []
            randomindex = randrange(len(kinglists[0]))
            img = image.load_img('/content/drive/My Drive/Emerging Technologies/Assignment 4/images/'+kinglists[0][randomindex],target_size=(224, 224)) #read in image
            img = image.img_to_array(img)
            img = preprocessinput(img)
            
            #create labels
            gender.append(kinglists[1][randomindex]) # gender
            area.append(np.array(kinglists[2][randomindex])) # area
            fight.append(np.array(kinglists[3][randomindex])) # fighting
            align.append(np.array(kinglists[4][randomindex])) # align
            color_list.append(np.array(kinglists[5][randomindex])) # color_list
            
            images.append(img)
            
        labels = [np.array(gender),np.array(area),np.array(fight),np.array(align), np.array(color_list)]
     
        if aug is not None:
            (images, labels) = next(aug.flow(np.array(images),labels, batch_size=bs))
        
        yield np.array(images),  labels 

# construct the training image generator for data augmentation
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15,
                         width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15,
                         horizontal_flip=True, fill_mode="nearest")

trainlists = [X_train, gender_train, area_train, fighting_style_train, align_train, color_list_train]
testlists = [X_test, gender_test, area_test, fighting_style_test, align_test, color_list_test]

trainGenerated = image_generator(trainlists, BS, 
                               mode="train", aug=None)
testGenerated = image_generator(testlists, BS, 
                              mode="train", aug=None)

In [9]:
# Color Images, Multi-Label Targets
from keras.optimizers import SGD, Adam

losslist = ["categorical_crossentropy",'categorical_crossentropy',
'categorical_crossentropy','categorical_crossentropy',
'binary_crossentropy']

matrics = {'gender': 'accuracy','area': 'accuracy',
               'fighting_style': 'accuracy','align': 'accuracy',
               'color': 'accuracy'}
dd = 0.0
import math
def stepdecay(epoch):
   initiallrate = 0.01
   drop = 0.5
   epochsdrop = 5.0
   lrate = initiallrate * math.pow(drop,  
           math.floor((1+epoch)/epochsdrop))
   return lrate

def multimodel(losslist,matrics,dd):
    
    basemodel = VGG19(weights='imagenet', include_top=False)

    #freeze all the layers
    for layer in basemodel.layers[:]:
       layer.trainable = False

    
    modelinput = Input(shape=(224, 224, 3))
    x = basemodel(modelinput)
    x = GlobalAveragePooling2D()(x)
    
    # fully-connected layer
    x = Dense(256, activation='relu')(x)
    x = Dropout(dd)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(dd)(x)
    # pass fully connected block output different model heads
    y1 = Dense(128, activation='relu')(x)
    y1 = Dropout(dd)(y1)
    y1 = Dense(64, activation='relu')(y1)
    y1 = Dropout(dd)(y1)
    
    y2 = Dense(128, activation='relu')(x)
    y2 = Dropout(dd)(y2)
    y2 = Dense(64, activation='relu')(y2)
    y2 = Dropout(dd)(y2)
    
    y3 = Dense(128, activation='relu')(x)
    y3 = Dropout(dd)(y3)
    y3 = Dense(64, activation='relu')(y3)
    y3 = Dropout(dd)(y3)

    y4 = Dense(128, activation='relu')(x)
    y4 = Dropout(dd)(y4)
    y4 = Dense(64, activation='relu')(y4)
    y4 = Dropout(dd)(y4)
    
    y5 = Dense(128, activation='relu')(x)
    y5 = Dropout(dd)(y5)
    y5 = Dense(64, activation='relu')(y5)
    y5 = Dropout(dd)(y5)
    
    #final output layers
    y1 = Dense(gender_nodes, activation='softmax',name= 'gender')(y1)
    y2 = Dense(area_nodes, activation='softmax',name= 'area')(y2)
    y3 = Dense(fighting_nodes, activation='softmax',name= 'fighting_style')(y3)
    y4 = Dense(align_nodes, activation='softmax',name= 'align')(y4)
    y5 = Dense(color_list_nodes, activation='sigmoid',name= 'color')(y5)
    
    model = Model(inputs=modelinput, outputs=[ y1, y2, y3, y4, y5])
    
    model.compile(loss=losslist, optimizer=SGD(lr=0.01,momentum=0.9), metrics=matrics)

    return model

multimodel = multimodel(losslist,matrics,dd)

multimodel.summary()

80142336/80134624 [==============================] - 1s 0us/step
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
vgg19 (Functional)              (None, None, None, 5 20024384    input_2[0][0]                    
__________________________________________________________________________________________________
global_average_pooling2d (Globa (None, 512)          0           vgg19[0][0]                      
__________________________________________________________________________________________________
dense (Dense)                   (None, 256)          131328      global_average_pooling2d[0][0]   
______________________

In [10]:
print(multimodel.metrics_names)


[]


In [11]:
checkpoint = ModelCheckpoint('models/best_run5_smaller.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')
lrate = LearningRateScheduler(stepdecay)
callbackslist = [checkpoint,lrate]
multimodel.fit_generator(trainGenerated,steps_per_epoch=len(X_train) // BS,
                    validation_data=testGenerated,
                    validation_steps=len(X_test) // BS,
                    epochs=EPOCHS,callbacks=callbackslist)




Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/50
22/22 [==============================] - ETA: 0s - loss: 7.9567 - gender_loss: 1.1386 - region_loss: 1.8827 - fighting_style_loss: 1.6972 - alignment_loss: 2.5016 - color_loss: 0.7365 - gender_accuracy: 0.5256 - region_accuracy: 0.4347 - fighting_style_accuracy: 0.4119 - alignment_accuracy: 0.2926 - color_accuracy: 0.1506
Epoch 00001: val_loss improved from inf to 5.89767, saving model to models/best_run5_smaller.h5
22/22 [==============================] - 51s 2s/step - loss: 7.9567 - gender_loss: 1.1386 - region_loss: 1.8827 - fighting_style_loss: 1.6972 - alignment_loss: 2.5016 - color_loss: 0.7365 - gender_accuracy: 0.5256 - region_accuracy: 0.4347 - fighting_style_accuracy: 0.4119 - alignment_accuracy: 0.2926 - color_accuracy: 0.1506 - val_loss: 5.8977 - val_gender_loss: 0.6413 - val_region_loss: 1.4047 - val_fighting_style_loss: 0.9363 - val_alignment_loss: 2.2203 - val_color_loss: 0.6951 - val_